In [ ]:
import tensorflow as tf 
import numpy as np 
import matplotlib.pyplot as plt 
from  tensorflow.keras.datasets import mnist   

In [ ]:
data=mnist 
#from keras.datasets import mnist
num_features= 784
num_classes = 10
(x_train,y_train),(x_test,y_test)=data.load_data()


In [ ]:
x_train,x_test=np.array(x_train,np.float32),np.array(x_test,np.float32)
x_train,x_test=x_train.reshape([-1,num_features]),x_test.reshape([-1,num_features])
x_train,x_test=x_train/255.,x_test/255.

In [ ]:
learning_rate=0.001
batch_size= 250
training_steps= 3000
display_step= 100
n_hidden= 512

In [ ]:
train_data=tf.data.Dataset.from_tensor_slices((x_train,y_train))
train_data=train_data.repeat().shuffle(60000).batch(batch_size).prefetch(1)

In [ ]:
random_normal= tf.initializers.RandomNormal()
weights= {
    'h': tf.Variable(random_normal([num_features,n_hidden])),
    'out': tf.Variable(random_normal([n_hidden,num_classes]))
}
biases={
     'h': tf.Variable(tf.zeros([n_hidden])),
    'out': tf.Variable(tf.zeros([num_classes]))
}

In [ ]:
def neural_net(input_data):
    hidden_layer= tf.add(tf.matmul(input_data,weights['h']),biases['h'])
    hidden_layer=tf.nn.sigmoid(hidden_layer)
    out_layer=tf.add(tf.matmul(hidden_layer,weights['out']),biases['out'])
    return (tf.nn.softmax(out_layer))


In [ ]:
def cross_entropy(y_pred,y_true):
    y_true = tf.one_hot(y_true,depth=num_classes)
    y_pred = tf.clip_by_value(y_pred,1e-9,1.)
    loss = -tf.reduce_sum(y_true * tf.math.log(y_pred))
    return (tf.reduce_mean(loss))



In [ ]:
optimizer=tf.keras.optimizers.SGD(learning_rate)
def run_optimization(x,y):
    with tf.GradientTape() as g:
        pred=neural_net(x)
        loss=cross_entropy(pred,y)
    trainable_variables=list(weights.values())+list(biases.values())
    gradients=g.gradient(loss,trainable_variables)
    optimizer.apply_gradients(zip(gradients,trainable_variables))
    

In [ ]:
def accuracy(y_pred,y_true):
    correct_pred = tf.equal(tf.argmax(y_pred,1),tf.cast(y_true,tf.int64))
    return(tf.reduce_mean(tf.cast(correct_pred,tf.float32),axis=-1))


In [ ]:
for step,(batch_x,batch_y) in enumerate(train_data.take(training_steps),1):
    #print(type(batch_x), type(batch_y))
    
    run_optimization(batch_x,batch_y)
    if step % display_step == 0:
        pred = neural_net(batch_x)
        loss = cross_entropy(pred,batch_y)
        acc = accuracy(pred,batch_y)
        print ("Training Epoch : %i, Loss : %f, Accuracy : %f" %(step,loss,acc))
    


In [ ]:
pred = neural_net(x_test)
acc = accuracy(pred,y_test)
print ("Testing Accuracy : %f" % acc)


In [ ]:
%matplotlib inline 
def display_sample(num):
    label = y_train[num]
    image = x_train[num].reshape([28,28])
    plt.title("Sample : %d Label : %d"% (num,label))
    plt.imshow(image, cmap = plt.get_cmap('gray_r'))
    plt.show()
display_sample(400)  

In [ ]:
images=x_train[0].reshape([1,784])
for i in range(1,500):
    images=np.concatenate((images,x_train[i].reshape([1,784])))
plt.imshow(images,cmap=plt.get_cmap('gray_r'))
plt.show()


In [ ]:
n_images=200
test_images=x_test[:n_images]
test_labels=y_test[:n_images]
predictions = neural_net(test_images)

for i in range(n_images):
    model_pred = np.argmax(predictions.numpy()[i])
    if (model_pred != test_labels[i]):
        plt.title("True Label : %d, Prediction : %d" %(test_labels[i],model_pred))
        plt.imshow(np.reshape(test_images[i],[28,28]),cmap = 'gray_r')
        plt.show()
